In [8]:
import sqlite3
import pandas as pd

# Pick the folders with .dbs to look through
import os
from tkinter import Tk, filedialog

def pick_db_folder():
    # Create a hidden Tkinter root window
    root = Tk()
    root.withdraw()

    # Ask user to select a folder
    db_path = filedialog.askdirectory(title='Select folder with VGS .db files')
    root.destroy()  # Close the Tk window properly

    # If user cancels
    if not db_path:
        print("No folder selected.")
        return []

    print('Selected db_path:', db_path)

    # Find .db files
    db_files = [f for f in os.listdir(db_path) if f.endswith('.db')]
    print('Found .db files:', db_files)

    # If no .db files found
    if not db_files:
        print("No .db files found in this folder.")
        return []

    # Return full paths
    db_files = [os.path.join(db_path, f) for f in db_files]
    return db_files


In [2]:
## functions for Hex guids
import re

def convert_to_hex(uuid_str):
    # Remove braces and hyphens
    guid = re.sub(r"[{}-]", "", uuid_str)

    # Reorder bytes (same as your R function)
    hex_guid = (
        guid[6:8] + guid[4:6] + guid[2:4] + guid[0:2] +
        guid[10:12] + guid[8:10] +
        guid[14:16] + guid[12:14] +
        guid[16:]
    ).lower()

    return f"X'{hex_guid}'"

def convert_from_hex(hex_str):
    # Remove X' and trailing '
    guid = hex_str[2:-1]

    # Reverse the byte swapping
    reverse_hex = (
        guid[6:8] + guid[4:6] + guid[2:4] + guid[0:2] + "-" +
        guid[10:12] + guid[8:10] + "-" +
        guid[14:16] + guid[12:14] + "-" +
        guid[16:20] + "-" +
        guid[20:]
    ).lower()

    return reverse_hex


**Missing Data from Fall 2025 V-V with Caleb, Ashley, Andrew, ect. - Walker Basin Allotment**

*USFS | Region 03 | Coconino National Forest | Red Rock Ranger District | Walker Basin Allotment*

-Transect 1, Cedar Flats C C11 (117 C11 -v - T1) 1d7dda48-732d-4358-94d5-00f76c24e70b

-Transects 1 - 5 and Tenth Acre, Cedar Flats F C17 ? (2 events) f8e9d877-5125-4715-8bfa-29c69149e3ad

-Tenth Acre, Twin Buttes C21 (tenth acre - 10/26/2025?) 6329351b-4353-404f-ad12-ee1927c2d7e2

-Transects 4 - 5, Cedar Flats D C5 (T4/5 - 10/28/2025) 0b671881-afe9-4711-b410-0dcff0e090bb

-Transects 4 -5, Cedar Flats A C8 (T4/5 - 10/28/2025) 3d6e22d1-7c85-4193-8676-d5e739d648f3


*To do this you need a list of pk_sites!!!*

These sites are where data may be missing. The function below will check these sites for data for the list of sites for as many databases as you want (.db's). You must also select a year you are looking for and if you want to check for individual transects.

In [3]:
# list of site pk_sites to seach each database for 
guids = [
    '1d7dda48-732d-4358-94d5-00f76c24e70b',
    'f8e9d877-5125-4715-8bfa-29c69149e3ad',
    '6329351b-4353-404f-ad12-ee1927c2d7e2',
    '0b671881-afe9-4711-b410-0dcff0e090bb',
    '3d6e22d1-7c85-4193-8676-d5e739d648f3'
]

# convert the guids to hex format for searching in the databases
converted_list =[]

# loop through the guids and convert each one to hex format, then store in a list
for i in guids:
    converted = convert_to_hex(i)
    converted_list.append(converted)

pd.DataFrame(converted_list, columns=["site_guids"])

,site_guids
0,X'48da7d1d2d73584394d500f76c24e70b'
1,X'77d8e9f8255115478bfa29c69149e3ad'
2,X'1b35296353434f40ad12ee1927c2d7e2'
3,X'8118670be9af1147b4100dcff0e090bb'
4,X'd1226e3d857c93418676d5e739d648f3'


In [4]:
converted_list[0]

"X'48da7d1d2d73584394d500f76c24e70b'"

In [ ]:

# e.g., year = 2025, site  = converted_list[0] (or loop through the list of sites)
## want to check for missing data specifics
def check_for(site_list, year, connection, check_transects=True):

    for site in site_list:
        if check_transects:
            # construct the SQL query to retrieve data for the specified site and year
            query = f"""select DISTINCT SiteID, Protocol.Date, Protocol.ProtocolName, Sample.Transect from  Protocol
            INNER JOIN EventGroup ON EventGroup.FK_Protocol = Protocol.PK_Protocol
            INNER JOIN Event ON Event.FK_EventGroup = EventGroup.PK_EventGroup
            INNER JOIN Site ON Site.PK_Site = Event.FK_Site
            JOIN SiteClassLink ON SiteClassLink.FK_Site = Site.PK_Site
            JOIN SiteClass ON SiteClass.PK_SiteClass = SiteClassLink.FK_SiteClass
            INNER JOIN AncestryCombinedPath ON AncestryCombinedPath.PK_Site = Site.PK_Site
            INNER JOIN Sample on Sample.FK_Event = Event.PK_Event
            where Site.PK_Site = {site}
            and Protocol.Date Like '%{year}%'"""
        else:
            query = f"""select DISTINCT SiteID, Protocol.Date, Protocol.ProtocolName from  Protocol
            INNER JOIN EventGroup ON EventGroup.FK_Protocol = Protocol.PK_Protocol
            INNER JOIN Event ON Event.FK_EventGroup = EventGroup.PK_EventGroup
            INNER JOIN Site ON Site.PK_Site = Event.FK_Site
            JOIN SiteClassLink ON SiteClassLink.FK_Site = Site.PK_Site
            JOIN SiteClass ON SiteClass.PK_SiteClass = SiteClassLink.FK_SiteClass
            INNER JOIN AncestryCombinedPath ON AncestryCombinedPath.PK_Site = Site.PK_Site
            where Site.PK_Site = {site}
            and Protocol.Date Like '%{year}%'""" 

        # execute the query and fetch the 
        df = pd.read_sql_query(query, connection)

        print(f"\n=== Results for GUID {site} (Year {year}) ===") 
        
        if df.empty:
            print("No data found.") 
        else:
            site_id = df['SiteID'].iloc[0]
            print(f"SiteID: {site_id}") 
            print(df)



In [14]:
# get folders
db_files = pick_db_folder()

# go through each .db in dbfiles
for db_file in db_files:
    conn = sqlite3.connect(db_file)
    print(f"\nChecking database: {db_file} (db_files index: {db_files.index(db_file)})")
    check_for(site_list=converted_list,
              year=2025,
              connection=conn,
              check_transects=True)
    conn.close()

Selected db_path: C:/Users/tsgil/OneDrive/Documents/VGS - R/VGS-python/databases/older V-V
Found .db files: ['VGS50.db', 'VGS50_Range_Caleb_10_26_2025_18_9_28_293.db', 'VGS50_Range_Caleb_10_26_2025_19_49_4_6.db', 'VGS50_Range_Caleb_10_28_2025_18_28_29_57.db', 'VGS_20231013_142206.db', 'VGS_20241008_101414.db', 'VGS_20251113_130644.db']

Checking database: C:/Users/tsgil/OneDrive/Documents/VGS - R/VGS-python/databases/older V-V\VGS50.db (db_files index: 0)

=== Results for GUID X'48da7d1d2d73584394d500f76c24e70b' (Year 2025) ===
No data found.

=== Results for GUID X'77d8e9f8255115478bfa29c69149e3ad' (Year 2025) ===
No data found.

=== Results for GUID X'1b35296353434f40ad12ee1927c2d7e2' (Year 2025) ===
No data found.

=== Results for GUID X'8118670be9af1147b4100dcff0e090bb' (Year 2025) ===
No data found.

=== Results for GUID X'd1226e3d857c93418676d5e739d648f3' (Year 2025) ===
No data found.

Checking database: C:/Users/tsgil/OneDrive/Documents/VGS - R/VGS-python/databases/older V-V\VG

Checking a new set of folders...

# get folders
db_files = pick_db_folder()

# go through each .db in dbfiles
for db_file in db_files:
    conn = sqlite3.connect(os.path.join(db_path, db_file))
    print(f"\nChecking database: {db_file} (db_files index: {db_files.index(db_file)})")
    check_for(site_list=converted_list,
              year=2025,
              connection=conn,
              check_transects=True)
    conn.close()